Ten skrypt umożlia dokonanie pomiaru składowych RGB oraz zapisanie ich w postaci tablicy numpy (numpy - blibioteka pomocniczna, przydatna przy pomiarach, działaniach na macierzach itp.). 


Pomiary są odczytywane z portu szeregowego (domyślnie /dev/ttyUSB0 na linux'ie, zmiana możlia poprzez wpisanie innego portu bezpośrednio w skrypcie). Następnie są parsowane, a ostateczna forma to tablica wektorów zawierających pomiar w kolejności [R, G, B]. 

Ilość pomiarów uwarunkowana jest argumentem w funkcji measureRGB(iteratoins), gdzie iteratoins może przyjąć dowolną liczbę całkowitą, lub wartość "inf". W trybie nieskończonego pomiaru nie ma możliwości zapisu danych pomiarowych. 

Wartości składowych nie są przetworzone i stanowią wyjście czujnika TCS3200. Czujnik ten zamienia prąd na fotodiodach (RGB) na częstotliwość. Przy użyciu mikrokontolera mierzony jest natomiast okres, więc wynik jest odwrotnie proporcjonalny do rzeczywistych wartosci składowych RGB.  

In [20]:
def parseRGB_data(RGB_string):
    RGB_string = RGB_string.split('\r')
    for line in RGB_string:
        if not line == '':
            if line[0] == 'R':
                R_meas = int(line[1:len(line)])
            if line[0] == 'G':
                G_meas = int(line[1:len(line)])
            if line[0] == 'B':
                B_meas = int(line[1:len(line)])
    if R_meas and G_meas and B_meas:
        RGB_meas = [R_meas, G_meas, B_meas]
        return RGB_meas
    
def saveMeasurements(RGB_meas_list):
    if isinstance(RGB_meas_list, list):
        RGB_meas = np.asarray(RGB_meas_list)
        np.save(input("Type RGB measurements file name: "), RGB_meas)
    else:
        print("That's not a list!")
    
    
def measureRGB(iteratoins):
    
    # inform that programm has entered this function
    print("Otwieranie portu szeregowego: /dev/ttyUSB0")

    try:
        s = serial.Serial('/dev/ttyUSB0', 9600, timeout=2.5, rtscts=True)
        s.rts = False
        x = s.read()          # read one byte
        str_x = str(x, 'utf-8')
        if str_x == startByte:  # check if it's start bajt
            print("Rozpoczęto pomiar: ")
            if isinstance(iteratoins, str):
                if iteratoins == "inf":
                    while True:
                        RGB_bytes = s.read(300)    # read up to ten bytes (timeout)
                        RGB_string = str(RGB_bytes, 'utf-8')
                        print(RGB_string) 
            elif isinstance(iteratoins, int):
                RGB_measurements = []
                for i in range(iteratoins):
                    RGB_bytes = s.read(300)    # read up to ten bytes (timeout)
                    RGB_string = str(RGB_bytes, 'utf-8')
                    RGB_parsed = parseRGB_data(RGB_string)
                    print(RGB_parsed)
                    RGB_measurements.append(RGB_parsed)
                if RGB_measurements:
                    print("Saving file...")
                    saveMeasurements(RGB_measurements)
        print("Closing port")
        s.close()
    
    except Exception as e:
        print(e)
        s.close()

if __name__ == "__main__":
    import serial
    import sys
    import numpy as np
    
    # define start byte
    startByte = 'S';
    
    measureRGB(30)
    

Otwieranie portu szeregowego: /dev/ttyUSB0
Rozpoczęto pomiar: 
[1099, 1593, 959]
[342, 1593, 2521]
[1096, 805, 967]
[1102, 1591, 977]
[1113, 810, 1742]
[333, 813, 975]
[334, 1593, 970]
[337, 807, 966]
[1109, 1592, 1747]
[334, 1579, 972]
[337, 1591, 968]
[337, 1588, 952]
[347, 806, 960]
[346, 815, 960]
[1086, 817, 959]
[339, 1568, 1752]
[337, 812, 1739]
[337, 817, 960]
[336, 823, 1705]
[336, 1593, 963]
[341, 1585, 1737]
[1106, 827, 978]
[337, 1599, 959]
[338, 1590, 996]
[1115, 1598, 972]
[1103, 817, 967]
[338, 824, 1748]
[338, 830, 1007]
[343, 1599, 1763]
[340, 821, 961]
Saving file...
Type RGB measurements file name: Untable_Unknown_ALittleWarm
Closing port


In [16]:
import numpy as np

# Przyklad odczytania wynikow po sapisie
np.load("meas_30sek/Swiatlo_dzienne_Niemcewicza_wieczor.npy")

array([[207, 327, 259],
       [207, 327, 260],
       [207, 327, 260],
       [208, 327, 260],
       [208, 328, 260],
       [208, 328, 261],
       [208, 328, 261],
       [208, 329, 261],
       [209, 329, 261],
       [209, 329, 261],
       [209, 330, 262],
       [209, 330, 262],
       [209, 330, 262],
       [210, 330, 262],
       [210, 330, 263],
       [210, 331, 263],
       [210, 331, 263],
       [210, 332, 263],
       [211, 332, 263],
       [211, 332, 263],
       [211, 332, 263],
       [211, 333, 264],
       [212, 333, 264],
       [212, 333, 265],
       [212, 334, 265],
       [212, 334, 265],
       [212, 334, 265],
       [212, 334, 265],
       [213, 335, 266],
       [213, 335, 266]])

In [2]:
import sys
import glob
import serial


def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


if __name__ == '__main__':
    print(serial_ports())

['/dev/ttyUSB0', '/dev/ttyS0']
